In [2]:
import pandas as pd
import numpy as np
import io
import os
import re
from google.colab import drive
drive.mount('/content/drive')
datapath = '/content/drive/MyDrive/colab_data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
txt = pd.read_csv(os.path.join(datapath,'smr_data.csv'))

In [5]:
!pip install transformers
from transformers import pipeline
summarizer = pipeline("summarization")


     |████████████████████████████████| 2.5MB 4.0MB/s 
     |████████████████████████████████| 3.3MB 33.7MB/s 
     |████████████████████████████████| 901kB 39.7MB/s 


In [6]:
txt.head(3)

,Unnamed: 0,text,type,docid,summary
0,0,Ad sales boost Time Warner profit\n\nQuarterly...,business,business001,"b""TimeWarner said fourth quarter sales rose 2%..."
1,1,Dollar gains on Greenspan speech\n\nThe dollar...,business,business002,b'The dollar has hit its highest level against...
2,2,Yukos unit buyer faces loan claim\n\nThe owner...,business,business003,b'Yukos\' owner Menatep Group says it will ask...


In [7]:
print(summarizer(txt.text[0], max_length=130, min_length=30, do_sample=False))

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


[{'summary_text': ' Profits buoyed by one-off gains offset a profit dip at Warner Bros, and less users for AOL . For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from 2003 .'}]


In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")
# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer.encode("summarize: " + txt.text[0], return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

In [10]:
print(tokenizer.decode(outputs[0]))

<pad> ad sales boost time warner profit. firm says fourth quarter sales rose 2% to $11.1bn from $10.9bn. its own internet business, AOL, had mixed fortunes.</s>
